# 1. ResNet 이란?
 - 참고 : https://datascienceschool.net/view-notebook/958022040c544257aa7ba88643d6c032/

# 2. torchvision.models.resnet
 - resnet(18, 34, 50, 101, 152)를 만들 수 있다.
 - 3(RGB) * 224 * 224 입력을 기준으로 만들도록 되어 있다.
 - input size가 다른 경우 Resnet을 적용하려면 어떻게 해야될지 배워보자.

# 3. BasicBlock
### (1) BasicBlock 이란?
 - BasicBlock은 ResNet 18과 34를 위한 설계이다.

<br>

### (2) BasicBlock의 구조
#### 1) Input인 x가 들어온다.
#### 2) Input인 x가 Convolution layer(3x3, 64 -> 3x3, 64)들을 거쳐서 아웃풋 f(x)가 된다.
#### 3) 원래의 Input 값인 x(Identity)가 f(x)와 더해진다.
#### 4) relu를 통과시켜 output을 구한다.

# 4. BottleNeck
### (1) BottleNeck 이란?
 - BottleNeck은 ResNet 50, 101, 152를 위한 설계이다.
 
<br>

### (2) BottleNeck의 구조
#### 1) Input인 x가 들어온다.
#### 2) Input인 x가 Convolution layer(1x1, 64 -> 3x3, 64 -> 1x1, 256)들을 거쳐서 아웃풋 f(x)가 된다.
#### 3) 원래의 Input 값인 x(Identity)가 f(x)와 더해진다.
#### 4) relu를 통과시켜 output을 구한다.

# 5. ResNet 네트워크 뜯어보기

In [1]:
# 라이브러리 로드
import torch.nn as nn
import torch.utils.model_zoo as model_zoo

# 변수 선언
__all__ = ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152']

# 모델 url 선언
model_urls = {
  'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
  'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
  'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
  'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
  'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
}

# conv3x3 정의 : 3 * 3의 커널, 패딩 1로 정의
def conv3x3(in_planes, out_planes, stride=1):
  return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)

# conv1x1 정의 : 1 * 1의 커널, 패딩 0으로 정의
def conv1x1(in_planes, out_planes, stride=1):
  return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)
  
# BasicBlock
'''
 - BasicBlock은 ResNet 18과 34를 위한 설계이다.
 (1) Input인 x가 들어온다.
 (2) Input인 x가 Convolution layer(3x3, 64 -> 3x3, 64)들을 거쳐서 아웃풋 f(x)가 된다.
 (3) 원래의 Input 값인 x(Identity)가 f(x)와 더해진다.
 (4) relu를 통과시켜 output을 구한다.
'''
class BasicBlock(nn.Module):
  expansion = 1
  
  def __init__(self, inplanes, planes, stride=1, downsample=None):
    super(BasicBlock, self).__init__()
    self.conv1 = conv3x3(inplanes, planes, stride)
    self.bn1 = nn.BatchNorm2d(planes)
    self.relu = nn.ReLU(inplace=True)
    self.conv2 = conv3x3(planes, planes)
    self.bn2 = nn.BatchNorm2d(planes)
    self.downsample = downsample
    self.stride = stride
    
  def forward(self, x):
    # identity
    identity = x # Input인 x 값을 identity에 저장한다.
    
    # f(x)
    out = self.conv1(x) # 3 * 3 convolution, stride는 파라미터 값
    out = self.bn1(out)
    out = self.relu(out)
    out = self.conv2(out) # 3 * 3 convolution, stride는 1
    out = self.bn2(out)
    
    # downsample : f(x)는 convolution layer들을 거치기 때문에 크기가 작아질 수 있다. 그 경우 f(x)와 x의 크기 차이 때문에 f(x) + x가 불가능 하다. 이를 방지하기 위해 downsample을 하는 것이다.
    if self.downsample is not None: # downsample이 None이 아니면
      identity = self.downsample(x) # downsaple을 한다.
    
    # f(x) + x
    out += idenity
    
    # relu
    out = self.relu(out)
    
    return out

In [2]:
# BottleNeck
'''
 - BottleNeck은 ResNet 50, 101, 152를 위한 설계이다.
(1) Input인 x가 들어온다.
(2) Input인 x가 Convolution layer(1x1, 64 -> 3x3, 64 -> 1x1, 256)들을 거쳐서 아웃풋 f(x)가 된다.
(3) 원래의 Input 값인 x(Identity)가 f(x)와 더해진다.
(4) relu를 통과시켜 output을 구한다.
'''
class Bottleneck(nn.Module):
  expansion = 4
  
  def __init__(self, inplanes, planes, stride=1, downsample=None):
    super(Bottleneck, self).__init__()
    self.conv1 = conv1x1(inplanes, planes)
    self.bn1 = nn.BatchNorm2d(planes)
    self.conv2 = conv3x3(planes, planes, stride)
    self.bn2 = nn.BatchNorm2d(planes)
    self.conv3 = conv1x1(planes, planes * self.expansion)
    self.bn3 = nn.BatchNorm2d(planes * self.expansion)
    self.relu = nn.ReLU(inplace=True)
    self.downsample = downsample
    self.stride = stride
    
  def forward(self, x):
    # identity
    identity = x
    
    # f(x)
    out = self.conv1(x) # 1 * 1 convolution, stride는 1
    out = self.bn1(out)
    out = self.relu(out)
    out = self.conv2(out) # 3 * 3 convolution, stride는 파라미터 값
    out = self.bn2(out)
    out = self.relu(out)    
    out = self.conv3(out) # 1 * 1 convolution, stride는 1
    out = self.bn3(out)
    
    # downsample
    if self.downsample is not None:
      identity = self.downsample(x)
    
    # f(x) + x
    out += identity
    
    # relu
    out = self.relu(out)
    
    return out
  

In [3]:
# ResNet 네트워크 만들기
class ResNet(nn.Module):
  def __init__(self, block, layers, num_classes=1000, zero_init_residual=False): # 파라미터 값으로 block(BasicBlock or BottleNeck), class 수, zero_init_residual(True로도 해보기) 등 설정
    super(ResNet, self).__init__()
    
    self.inplanes = 64
    
    self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False) # input 3 * 224 * 224 -> output 64 * 112 * 112
    
    self.bn1 = nn.BatchNorm2d(64)
    self.relu = nn.ReLU(inplace=True)
    
    self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1) # input 64 * 112 * 112 -> output 64 * 56 * 56
    
    # ResNet 50 기준으로 레이어를 이해해 보자
    self.layer1 = self._make_layer(block, 64, layers[0]) # layers[0] = 3
    self.layer2 = self._make_layer(block, 128, layers[1], stride=2) # layers[1] = 4
    self.layer3 = self._make_layer(block, 256, layers[2], stride=2) # layers[2] = 6
    self.layer4 = self._make_layer(block, 512, layers[3], stride=2) # layers[3] = 3
    
    self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
    self.fc = nn.Linear(512 * block.expansion, num_classes)
    
    # weight initialize
    for m in self.modules(): # 모듈의 값을 하나씩 가져와서
      if isinstance(m, nn.Conv2d): # Conv이면
        nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
      elif isinstance(m, nn.BatchNrom2d): # BatchNorm이면
        nn.init.constant_(m.weight, 1)
        nn.init.constant_(m.bias, 0)
        
    # zero_init_residual
    if zero_init_residual: # zero_init_residual = True 일 때 실행 (성능이 0.2 ~ 0.3% 높아질 수 있다)
      for m in self.modules():
        if isinstance(m, Bottleneck):
          nn.init.constant_(m.bn3.weight, 0)
        elif isinstance(m, BasicBlock):
          nn.init.constant_(m.bn2.weight, 0)
  
  # ResNet 50 기준으로 이해해 보자
  def _make_layer(self, block, planes, blocks, stride=1): # BottleNeck, 64, 3 
    downsample = None
    
    # downsample은 f(x)와 x의 크기를 맞추거나, 채널 사이즈를 맞추기 위해 사용한다.
    if stride != 1 or self.inplanes != planes * block.expansion: # stride가 1이 아니거나, 'self.inplanes != planes * block.expansion(64 != 64 *4)' 라면, downsample을 실행하라.
      downsample = nn.Sequential( # downsample 진행
        conv1x1(self.inplanes, planes * block.expansion, stride), # conv1x1(64, 256, stride=1)
        nn.BatchNorm2d(planes * block.expansion), # BatchNorm2d(256)
      )
    
    layers = []
    layers.append(block(self.inplanes, planes, stride, downsample)) # layers.append(BottleNeck(64, 64, 1, downsample))
        
    self.inplanes = planes * block.expansion # self.inplanes = 256
    
    for _ in range(1, blocks): # for _ in range(1, 3)
      layers.append(block(self.inplanes, planes))
      
    return nn.Sequential(*layers)
    
    '''
    여기까지의 결과 ResNet 50의 첫번째 레이어에는 아래와 같이 쌓여있다는 걸 알 수 있다.
    [
      BottleNeck(64, 64, 1, downsample)
      BottleNeck(256, 64)
      BottleNeck(256, 64)
    ]
    
    '''

  def forward(self, x):
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)
    x = self.maxpool(x)
    
    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x)
    
    x = self.avgpool(x) # avgpool
    x = x.view(x.size(0), -1)
    x = self.fc(x)
    
    return x

In [4]:
# resnet18 정의
def resnet18(pretrained=False, **kwargs):
  model = ResNet(BasicBlock, [2, 2, 2, 2], **kwargs) # BasicBlock 구조 이용. 2 * (2 + 2 + 2 + 2) + 1(conv1) + 1(fc1) = 18 => ResNet18
  return model

# resnet50 정의
def resnet50(pretrained=False, **kwargs):
  model = ResNet(Bottleneck, [3, 4, 6, 3], **kwargs) # BottleNeck 구조 이용. 3 * (3 + 4 + 6 + 3) + 1(conv1) + 1(fc1) = 50 => ResNet50
  return model

# resnet152 정의
def resnet152(pretrained=False, **kwargs):
  model = ResNet(Bottleneck, [3, 8, 36, 3], **kwargs) # BottleNeck 구조 이용. 3 * (3 + 8 + 36 + 3) + 1(conv1) + 1(fc1) = 152 => ResNet152
  return model

# resnet 사용 방법
import torchvision.models.resnet as resnet # 라이브러리 로드
res = resnet.resnet50() # 모델
res


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 